In [26]:
import os
import sys

import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [56]:
# dataset
src = o3d.io.read_point_cloud("src.ply")
dst = o3d.io.read_point_cloud("dst.ply")

Xall = np.asarray(src.points)
Yall = np.asarray(dst.points)

const_buf = np.ones((Xall.shape[0], 1))
Xall = np.concatenate((const_buf, Xall), axis=1) # 1+ local position
Yalls = np.hsplit(Yall, Yall.shape[1]) # independent global coordinate 

[0.27405298 1.02095513 1.12868408]
[0.27405298] [1.02095513] [1.12868408]


In [25]:
# weighted least squares
W = np.eye(X.shape[0])

# y = x * beta
beta = np.linalg.inv(X.T * W * X) * (X.T * W * Y)
print(beta)

print(Y[0])
print(X[0] * beta)

[[ 9.84750e-01  1.73690e-01  4.00000e-05]
 [ 8.36850e-01  1.92539e+00  3.41930e-01]
 [ 5.94600e-02 -3.36860e-01  9.39720e-01]]
[[18.8106 17.6222 12.8169]]
[[18.8106 17.6222 12.8169]]


In [ ]:
# recursive least squares
# P = 